Big thanks to Pytorch Documentation for helping write the code

In [ ]:
MODEL_NAME = 'skewed_shrunk_bw_GAN'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
# !ls drive/MyDrive/cs7643/'DL Project'/generated_imgs
!rm -r generated
# !unzip drive/MyDrive/cs7643/'DL Project'/bagan_gp_results.zip -d ./generated

rm: cannot remove 'generated': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -r ./data

rm: cannot remove './data': No such file or directory


In [ ]:
!rm -r sample_data

In [ ]:
!unzip drive/MyDrive/cs7643/'DL Project'/skewed_shrunk_bw.zip -d .
!mv -f ./skewed/data data
!rm -r ./skewed

Streaming output truncated to the last 5000 lines.
 extracting: ./skewed/data/Training/pituitary/Tr-pi_1311.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0966.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0549.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_0761.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_0565.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0290.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_0520.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_0231.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0733.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_1146.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0492.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_0718.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0983.jpg  
 extracting: ./skewed/data/Training/pituitary/Tr-pi_0580.jpg  
  inflating: ./skewed/data/Training/pituitary/Tr-pi_0184.jpg  
  in

Load Generated Images into dataset

In [ ]:
# !unzip drive/MyDrive/cs7643/'DL Project'/diffusion_samples.zip -d ./generated
# !mv ./generated/diffusion_samples/* ./data/Training/pituitary/
!cp drive/MyDrive/cs7643/'DL Project'/generated_imgs/* ./data/Training/pituitary/

In [ ]:
data_transforms = {
    'Training': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Testing': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['Training', 'Testing']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['Training', 'Testing']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Training', 'Testing']}
class_names = image_datasets['Training'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


cuda:0


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Training', 'Testing']:
            if phase == 'Training':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Training'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'Training':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'Training':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'Testing' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
#Replaces final fully connected layer with an untrained one with 4 outputs
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

In [ ]:
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
# Train Model
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------
Training Loss: 1.1803 Acc: 0.5100
Testing Loss: 0.7681 Acc: 0.6968

Epoch 1/19
----------
Training Loss: 0.9631 Acc: 0.6104
Testing Loss: 0.7371 Acc: 0.7069

Epoch 2/19
----------
Training Loss: 0.8943 Acc: 0.6382
Testing Loss: 0.5993 Acc: 0.7853

Epoch 3/19
----------
Training Loss: 0.7822 Acc: 0.6891
Testing Loss: 0.5813 Acc: 0.7323

Epoch 4/19
----------
Training Loss: 0.7173 Acc: 0.7143
Testing Loss: 0.5667 Acc: 0.7786

Epoch 5/19
----------
Training Loss: 0.6629 Acc: 0.7401
Testing Loss: 0.4235 Acc: 0.8470

Epoch 6/19
----------
Training Loss: 0.6257 Acc: 0.7616
Testing Loss: 0.4713 Acc: 0.8286

Epoch 7/19
----------
Training Loss: 0.4899 Acc: 0.8135
Testing Loss: 0.3533 Acc: 0.8675

Epoch 8/19
----------
Training Loss: 0.4769 Acc: 0.8213
Testing Loss: 0.3358 Acc: 0.8732

Epoch 9/19
----------
Training Loss: 0.4529 Acc: 0.8262
Testing Loss: 0.3423 Acc: 0.8782

Epoch 10/19
----------
Training Loss: 0.4666 Acc: 0.8175
Testing Loss: 0.3222 Acc: 0.8849

Epoch 11/

In [ ]:
torch.save(model_ft.state_dict(), f'./{MODEL_NAME}.pth')

In [ ]:
!cp skewed_shrunk_bw_GAN.pth drive/MyDrive/cs7643/'DL Project'/skewed_shrunk_bw_GAN.pth

In [ ]:
# import shutil
# shutil.copyfile(f'./{MODEL_NAME}.pth', f"drive/MyDrive/cs7643/'DL Project'/{MODEL_NAME}.pth")